In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
#set path to import my_tools.py
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from datetime import datetime
import glob
%matplotlib qt


In [24]:
#set path change accordingly
path = '/media/onno/Algemeen/Thesis/'
#List specific variable data
file_gfs = 'gefsrf2_env_wledit2000-10000_latavg_v300_control0-252h_6hourly_2x2_dec84-nov19.nc'
file_era = 'era51_env_wledit2000-10000_latavg_v300_79-19_6hourly_anom_from_smoothed04_clim_smoothed.nc'
#open xarray datasets
gfs = xr.open_dataset(path+file_gfs,decode_times=False)
era = xr.open_dataset(path+file_era,decode_times=False)
#change time values to normal datetime values
gfs['time']=my_tools.convert_date_gefs_r(gfs.time.values)
era['time']=my_tools.convert_date_era_r(era.time.values)

In [25]:
#Select only northern hemisphere and for GFS reforecast data only select data for 24h intervals
gfs_nh = gfs.sel(lat=slice(90,0),lead=gfs.lead.values[[bool(i) for i in gfs.lead%24==0]])
era_nh = era.sel(lat=slice(90,0))

In [26]:
t1 = datetime.now()
#loop over all lead days
for i in range(11):
    forecast = gfs_nh.sel(time=[i for i in pd.date_range('1984-12-1','2019-11-{}'.format(30-i))],lead=i*24)
    reanalysis = era_nh.sel(time=[i for i in pd.date_range('1984-12-{}'.format(1+i),'2019-11-30')])
#     assert(forecast.time.values[0]==reanalysis.time.values[0]+pd.Timedelta(i,'d'))
    diff = forecast - reanalysis
    diff_p25 = diff.quantile(q=.25,dim='time')
    diff_p75 = diff.quantile(q=.75,dim='time')
    diff_inter_quan = diff_p75 - diff_p25
#     forecast_q25 = forecast.quantile(q=0.25,dim='time')
#     forecast_q75 = forecast.quantile(q=0.75,dim='time')
#     reanalysis_q25 = reanalysis.quantile(q=0.25,dim='time')
#     reanalysis_q75 = reanalysis.quantile(q=0.75dim='time')
    diff_inter_quan.to_netcdf('/media/onno/Algemeen/Thesis/GFS_inter_quan/GFS_phasespeed_interquantile_range_day_{:02d}'.format(i))
t2 = datetime.now()
print(t2-t1)


/home/onno/miniconda3/envs/thesis/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1389: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/home/onno/miniconda3/envs/thesis/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1389: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/home/onno/miniconda3/envs/thesis/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1389: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/home/onno/miniconda3/envs/thesis/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1389: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/home/onno/miniconda3/envs/thesis/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1389: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/home/onno/miniconda

0:30:38.390938
